# Part 1: API Overview

In [ ]:
from __future__ import absolute_import, division, print_function
import logging
import sys
import os
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))
from clipper_admin import ClipperConnection, DockerContainerManager

Start Clipper on Docker. Link to API Docs.

In [ ]:
clipper_conn = ClipperConnection(DockerContainerManager("localhost"))
clipper_conn.start_clipper()

Check Docker and see that there are now 3 Docker containers running.

In [1]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


At its most basic, Clipper is a function server. To start with, lets deploy a very simple function to Clipper. We'll start with the classic Big Data hello-world example: word count.

First, we'll define our word count function. Clipper may provide multiple inputs to a function at once, because many machine learning models can take advantage of data parallelism when performing inference to improve performance. Because of this, we define a second function that takes a list of documents as an argument and returns the word count of each document.

In [ ]:
def count_words(text):
    words = text.split()
    return len(words)

# The function you deploy to Clipper must take a list of inputs, so we
# wrap our core logic in loop.
def count_words_in_docs(docs):
    counts = []
    for d in docs:
        count = str(count_words(d))
        counts.append(count)
    return counts

In [ ]:
example_one = "Hello world. Foo bar baz."
example_two = "I'm learning so much at the inaugural RISE Camp."

print("There are %d words in the first example." % count_words(example_one))
count_words_in_docs([example_one, example_two])

You can now deploy this function directly to Clipper.

In [ ]:
from clipper_admin.deployers import python as python_deployer
python_deployer.create_endpoint(clipper_conn, name="wordcount", input_type="string", func=count_words_in_docs)

Try querying the newly created application with curl. You can run this directly in the Jupyter notebook or locally on your laptop.

In [ ]:
!curl -X POST --header "Content-Type:application/json" -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' 127.0.0.1:1337/wordcount/predict


Inspect Clipper's internal metrics

In [ ]:
clipper_conn.inspect_instance()

In [ ]:
clipper_conn.get_all_apps(verbose=True)

In [ ]:
clipper_conn.get_all_models(verbose=True)

In [ ]:
!docker ps

Now let's update our word count function to count word frequencies instead of just the total number of words in a document.

In [ ]:
import json
import string
def word_freq(text):
    # Convert to lower case
    text = text.lower()
    text = text.translate(None, string.punctuation)
    words = text.split()
    counts = {}
    for w in words:
        if w in counts:
            counts[w] += 1
        else:
            counts[w] = 1
    return json.dumps(counts)
    
# The function you deploy to Clipper must take a list of inputs, so we
# wrap our core logic in loop.
def count_word_freqs(docs):
    counts = []
    for d in docs:
        count = word_freq(d)
        counts.append(count)
    return counts

In [ ]:
word_freq_example = "Hello world. This is an example sentence. This is another sentence. World world world."
count_word_freqs([word_freq_example])

Now deploy our new and improved word counter to Clipper as a new version of the "wordcount" model. If we look above at the output of `clipper_conn.get_all_models` we can see that the currently deployed version is version "1". Let's deploy this new function as version "2".

In [ ]:
python_deployer.deploy_python_closure(clipper_conn, name="wordcount", version="2", input_type="strings", func=count_word_freqs)

In [ ]:
!curl -X POST --header "Content-Type:application/json" -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' 127.0.0.1:1337/wordcount/predict

Now if you decide that you actually liked the previous version better (or more realistically discover a bug), you can roll back to an earlier version.

In [ ]:
clipper_conn.set_model_version(name="wordcount", version="1")

In [ ]:
!curl -X POST --header "Content-Type:application/json" -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' 127.0.0.1:1337/wordcount/predict

# Part 2: Example Application - Birds vs Airplanes

In part 2 of this exercise, you will build a real machine application that uses computer vision models to classify images.

You will create an application that labels images as pictures of either birds or planes. You will use the CIFAR-10 dataset as the source of these images.

These images have already been downloaded and are available locally at `~/cifar`.



## Load Cifar

The first step in building any application, using machine-learning or otherwise, is to understand the application requirements. Load the dataset into the notebook so you can examine it and better understand the dataset you will be working with. The `cifar_utils` library provides several utilities for working with CIFAR data – we will make use of one of them here.

In [ ]:
import cifar_utils

cifar_loc = os.path.expanduser("cifar/")
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))
train_x, train_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_train.data", norm=True))
raw_x, raw_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=False))

Take a look at the data you've loaded. The size and blurriness of these photos should give you a better understanding of the difficulty of the task you will ask of your machine learning models! If you'd like to see more images, increase the number of rows of images displayed -- the last argument to the function -- to a number greater than 2.

In [ ]:
%matplotlib inline
cifar_utils.show_example_images(raw_x, raw_y, 2)

# Create an application

In order to query Clipper for predictions, you need to create an application. Each application specifies a name, the query input datatype, the selection policy, and a latency service level objective. Once you register an application with Clipper, the system will create a REST endpoint for handling prediction requests.

By associating the query interface with a specific application, Clipper allows frontend developers the flexibility to have multiple applications running in the same Clipper instance. Applications can request predictions from any model in Clipper. This allows a single Clipper instance to serve multiple machine-learning applications. It also provides a convenient mechanism for beta-testing or incremental rollout by creating experimental and stable applications for the same set of queries.

For this tutorial, you will create an application named "cifar_demo". Note that Clipper allows you to create the application before deploying the models. Clipper will be moving to a label-based model specification mechanism soon, so that in the future you won't have to explicitly enumerate all the models you want to query up front.

Registering the `cifar-demo` application with Clipper will have the following effect on your setup: <img src="img/register_app.png" style="width: 500px;"/>

> *Don't worry if this command seems to take a long time. Before starting Clipper, the Docker containers must be downloaded from Docker Hub. These containers are fairly large and may take awhile to download depending on the speed of your internet connection.*

In [ ]:
app_name = "cifar-demo"
# If the model (which we will later link to our application) doesn't
# return a prediction in time, predict label 0 (bird) by default
default_output = "0"

clipper_conn.register_application(
    app_name,
    "doubles",
    default_output,
    slo_micros=50000)

Now when you list the applications registered with Clipper, you should see the newly registered "cifar-demo" application show up!

In [ ]:
clipper_conn.get_all_apps(verbose=True)

# Start serving

Now that you have registered an application, you can start querying the application for predictions. In this case,
Clipper has created a REST endpoint for your application at:
```
http://localhost:1337/cifar_demo/predict
```

You will now start querying Clipper with a simple Python frontend app that computes the average accuracy of the responses after every 100 requests and updates a plot of the results with every iteration.

This diagram shows how the accuracy plot is receiving its test predictions: <img src="img/serve_predictions.png" style="width: 500px;"/>

Go to the [query_cifar](query_cifar.ipynb) notebook to start the app.

# Train Logistic Regression Model

When tackling a new problem with machine learning, it's always good to start with simple models and only add complexity when needed. Start by training a logistic regression binary classifier using [Scikit-Learn](http://scikit-learn.org/). This model gets about 68% accuracy on the offline evaluation dataset if you use 10,000 training examples. It gets about 74% if you use all 50,000 examples.

In [ ]:
from sklearn import linear_model as lm 
def train_sklearn_model(m, train_x, train_y):
    m.fit(train_x, train_y)
    return m
lr_model = train_sklearn_model(lm.LogisticRegression(), train_x, train_y)
print("Logistic Regression test score: %f" % lr_model.score(test_x, test_y))

# Deploy Logistic Regression Model

While 68-74% accuracy on a CIFAR binary classification task is significantly below state of the art, it's already much better than the 50% accuracy your application yields right now by guessing randomly.

You can deploy your logistic regression model directly to Clipper without having to worry about how to serialize the model or integrate it with application code.

To deploy a model to Clipper, you must assign it a name ("sklearn_cifar"), a version (1), and then provide some metadata about the model itself. In this case, you are specifying that you want to run the model using the `sklearn_cifar_container` Docker image in the [Clipper repo](https://hub.docker.com/u/clipper/dashboard/) on Docker Hub. You can assign the model descriptive labels, and specify the input type that this model expects. Finally, you can specify how many *replicas* of the model (how many Docker containers) to launch. Adding more replicas increases the throughput of this model.

After completing this step, Clipper will be managing a new container in Docker with your model in it:
<img src="img/deploy_sklearn_model.png" style="width: 500px;"/>

> *Once again, because you are deploying a Docker image this command may take awhile to download the image. Thanks for being patient!*

In [ ]:

def classify_ufos(images):
    preds = lr_model.predict(images)
    return [str(p) for p in preds]

print("Predicted labels: {}".format(classify_ufos(test_x[0:3])))
print("Correct labels: {}".format(test_y[0:3]))


In [ ]:
model_name = "classify-ufos"
python_deployer.deploy_python_closure(clipper_conn, name=model_name, version="1", input_type="doubles", func=classify_ufos)

## Link your app to your model
To use your newly deployed model to generate predictions, it needs to be linked to your Clipper application. Let Clipper know that your `"cifar-demo"` app should use the `"classify-ufos"` model to serve predictions.

In the future, when you deploy new versions of the `"classify-ufos"` model, queries to your Clipper application will route to them.

In [ ]:
clipper_conn.link_model_to_app(app_name, model_name)

Now that you've deployed and linked your model to your app, go ahead and check back on your running frontend application in [query_cifar](query_cifar.ipynb). You should see the accuracy rise from around 50% to the accuracy of your SKLearn model (68-74%), without having to stop or modify your application at all!

# Load TensorFlow Model

To improve the accuracy of your application further, you will now deploy a TensorFlow convolutional neural network. This model takes a few hours to train, so you will download the trained model parameters rather than training it from scratch. This model gets about 88% accuracy on the test dataset.

We have provided a pre-trained TensorFlow model stored at `tf_cifar_model/cifar10_model_full`.

In [ ]:
import os
import tensorflow as tf
import numpy as np
tf_cifar_model_path = os.path.abspath("tf_cifar_model/cifar10_model_full")
tf_session = tf.Session('', tf.Graph())
with tf_session.graph.as_default():
    saver = tf.train.import_meta_graph("%s.meta" % tf_cifar_model_path)
    saver.restore(tf_session, tf_cifar_model_path)

def tensorflow_score(session, test_x, test_y):
    """
    NOTE: This predict method expects pre-whitened (normalized) images
    """
    logits = session.run('softmax_logits:0',
                           feed_dict={'x:0': test_x})
    relevant_activations = logits[:, [cifar_utils.negative_class, cifar_utils.positive_class]]
    preds = np.argmax(relevant_activations, axis=1)
    return float(np.sum(preds == test_y)) / float(len(test_y))
print("TensorFlow ConvNet test score: %f" % tensorflow_score(tf_session, test_x, test_y))

# Deploy TensorFlow Model

You can now deploy your TensorFlow neural network. Because TensorFlow models cannot be pickled, we cannot deploy them directly using a Python closure. Instead, we will serialize the model using the TensorFlow serialization API and deploy the serialized model directly to Clipper using a custom TensorFlow container.

After completing this step, Clipper will send queries to the newly-deployed TensorFlow model instead of the logistic regression Scikit-Learn model, improving the application's accuracy.
<img src="img/tf_replaces_sklearn_model.png" style="width: 600px;"/>

> *Once again, please patient while the Docker image is downloaded.*

In [ ]:
clipper_conn.build_and_deploy_model(
    name=model_name,
    version="2",
    input_type="doubles",
    model_data_path=os.path.abspath("tf_cifar_model"),
    base_image="clipper/tf_cifar_container:risecamp")


Check out the frontend again and see how the accuracy has gone.

Finally, when you are all done, you can stop Clipper and cleanup the docker containers.

In [ ]:
clipper_conn.stop_all()

In [ ]:
!docker ps